In [73]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pathlib
import ezodf
import pandas as pd

In [11]:
url = "https://www.data.gov.uk/dataset/5d5028ef-9918-4ab7-8755-81f3ad06f308/pesticide-residues-in-food"

In [12]:
def get_soup(url): #integrated function to return parsed HTML
    return bs(requests.get(url).content)

soup = get_soup(url)

In [13]:
# getting a list of the links that contain ".ods" filename extension
file_urls = [a.get('href') for a in soup.find_all('a', href=True) if (re.search('ods',(a.get('href'))))] 

In [38]:
# don't want repeated data, so take annual data if available, then quarterly up to most recent
ann_file_urls = [x for x in file_urls if 'annual' in x.lower()]

def return_years_available(urls):
    years = []
    for url in urls:
        match = re.search(r'\d{4}', url)
        if match:
            year = match.group()
            years.append(year)
    return set(years)
        
        
years_available = return_years_available(file_urls)        
years_with_ann = return_years_available(ann_file_urls)

years_need_quarterly = list(years_available - years_with_ann)

In [47]:
# most recent quarterly urls where annual data isn't available
qua_file_urls = [x for x in file_urls if any(y in x for y in years_need_quarterly) and 'quarterly' in x.lower()]

In [49]:
file_urls = ann_file_urls + qua_file_urls

In [51]:
file_names = [re.search('([A-Za-z_0-9.-]*\.ods)', url).group() for url in file_urls]

In [62]:
file_urls

['https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/2021_annual_data.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/2020_prif_Annual.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/2019_annual_data.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/2018_annual_data_v2.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/2017_annual_data.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/2016_annual_data.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/Q2_2022_quarterly_data.ods',
 'https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/PRIF/Q1_2022_quarterly_data.ods']

In [66]:
def get_path(file_name): 
    filetopath = pathlib.Path.cwd() / "data" / file_name
    return filetopath

In [68]:
def save_files_locally(file_urls, file_names):
    for url,file_name in zip(file_urls,file_names):
        # getting the raw data from each url
        response = requests.get(url, stream=True)
        filetopath = get_path(file_name)
        # using open method to open a file on system and write the contents
        with open(filetopath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    # using with open write to write response.content to local filepath 
                    f.write(chunk) 

In [69]:
# saves each file in the current directory, in a folder called "data"
save_files_locally(file_urls,file_names) 

In [140]:
def get_doc(filetopath):
    doc = ezodf.opendoc(filetopath)
    return doc

def sheet_names_indexes(doc):
    sheet_name_index = {sheet.name: i for i,sheet in enumerate(doc.sheets)}
    return sheet_name_index

def doc_to_sheet(doc,ind):
    sheet = doc.sheets[ind]
    return sheet

def get_len_of_doc(doc): #gets the number of sheets in the document
    count = len(doc.sheets)
    return count

def identify_categories(row): ### Could use this to put categories as their own column and then fill forward. 
    cells = [cell.value for cell in row]
    if cells[0] != None and all(x is None for x in cells[1:]):
        return True
    else:
        return False

def sheet_to_df(sheet):
    df_dict = {}
    
    for i, row in enumerate(sheet.rows()):
    #     row is a list of cells
    # #     assume the header is on the first row
        if i == 0:
            title = sheet.row(0)[0].value
        elif i == 1:
            # columns as lists in a dictionary
    #         print(cell.value)
    #         df_dict.update({cell.value:[]})
            df_dict = {cell.value:[] for cell in row}
            col_index = [col_title for col_title in df_dict.keys()]
        elif i > 1:
            if identify_categories(row): ###should this be outside the for loop?
                 ### want to do something useful with this but first get it to skipp any category rows
                pass
            for j,cell in enumerate(row):
                try:
                    df_dict[col_index[j]].append(cell.value)
                except:
                    pass
    return pd.DataFrame(df_dict)

In [141]:
# file_name = file_names[0]
dfs = []
cols = ['Sample ID', 'Date of Sampling', 'Description', 'Country of Origin',
        'Sampling Point', 'Address', 'Brand Name', 'Packer / Manufacturer / Importer', 
        'Pesticide residues found in mg/kg (MRL)', None]
for file in file_names:
    filetopath = get_path(file)
    doc = get_doc(filetopath)
    sheet_name_idx = sheet_names_indexes(doc)
    bna_idxs = [value for key, value in sheet_name_idx.items() if 'BNA' in key]
    for idx in bna_idxs:
        sheet = doc_to_sheet(doc, idx)
        df = sheet_to_df(sheet)
        try:
            df.columns = cols
        except:
            pass
        dfs.append(df)

2021_annual_data.ods
{'Summary': 0, 'Asparagus_GB_BNA': 1, 'Asparagus_GB_SUM': 2, 'Asparagus_NI_BNA': 3, 'Asparagus_NI_SUM': 4, 'Aubergine_GB_BNA': 5, 'Aubergine_GB_SUM': 6, 'Aubergine_NI_BNA': 7, 'Aubergine_NI_SUM': 8, 'Banana_GB_BNA': 9, 'Banana_GB_SUM': 10, 'Banana_NI_BNA': 11, 'Banana_NI_SUM': 12, 'Beans_GB_BNA': 13, 'Beans_GB_SUM': 14, 'Beans_NI_BNA': 15, 'Beans_NI_SUM': 16, 'Beef_GB_BNA': 17, 'Beef_GB_SUM': 18, 'Beef_NI_BNA': 19, 'Beef_NI_SUM': 20, 'Berries_GB_BNA': 21, 'Berries_GB_SUM': 22, 'Berries_NI_BNA': 23, 'Berries_NI_SUM': 24, 'Bread_GB_BNA': 25, 'Bread_GB_SUM': 26, 'Bread_NI_BNA': 27, 'Bread_NI_SUM': 28, 'Broccoli_GB_BNA': 29, 'Broccoli_GB_SUM': 30, 'Broccoli_NI_BNA': 31, 'Broccoli_NI_SUM': 32, 'Cheese_GB_BNA': 33, 'Cheese_GB_SUM': 34, 'Cheese_NI_BNA': 35, 'Cheese_NI_SUM': 36, 'Edible_Seeds_GB_BNA': 37, 'Edible_Seeds_GB_SUM': 38, 'Eggs_GB_BNA': 39, 'Eggs_GB_SUM': 40, 'Eggs_NI_BNA': 41, 'Eggs_NI_SUM': 42, 'Fish_NI_BNA': 43, 'Fish_NI_SUM': 44, 'Grapefruit_GB_BNA': 45, 'Gra

16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
2016_annual_data.ods
{'Apple_BNA': 0, 'Apple_SUM': 1, 'Apricots_BNA': 2, 'Apricots_SUM': 3, 'Beans_with_pods_BNA': 4, 'Beans_with_pods_SUM_': 5, 'Bread_BNA': 6, 'Bread_SUM': 7, 'Breakfast_Cereal_BNA': 8, 'Breakfast_Cereal_SUM': 9, 'Cabbage_BNA': 10, 'Cabbage_SUM': 11, 'Cashew_nuts_BNA': 12, 'Cashew_nuts_SUM': 13, 'Cheese_buffalo,_ewes,_goats_BNA': 14, 'Cheese_buffalo,_ewes,_goats_ST': 15, 'Cheese_processed_BNA': 16, 'Cheese_processed_SUM': 17, 'Cooked_meat_BNA': 18, 'Cooked_meat_SUM': 19, 'Fish_predator_BNA': 20, 'Fish_predator_SUM': 21, 'Fish_sea_BNA': 22, 'Fish_sea_SUM': 23, 'Gluten-free_food_BNA': 24, 'Gluten-free_food_SUM': 25, 'Grapefruit_BNA': 26, 'Grapefruit_SUM': 27, 'Grapes_BNA': 28, 'Grapes_SUM': 29, 'Honey_BNA': 30, 'Honey_SUM': 31, 'Infant_food_BNA': 32, 'Infant_food_SUM': 33, 'Jam_BNA': 34, 'Jam_SUM': 35, 'Leeks_BNA': 36, 'Leeks_SUM': 37, 'Lettuce_BNA': 38, 'Lettuce_SUM': 39, 'Milk_BN

In [179]:
df = pd.concat(dfs).reset_index(drop=True)

In [180]:
df = df.iloc[:,:9]

In [181]:
df.fillna(method='ffill', inplace=True) # forward filling the Nones 
df

,Sample ID,Date of Sampling,Description,Country of Origin,Sampling Point,Address,Brand Name,Packer / Manufacturer / Importer,Pesticide residues found in mg/kg (MRL)
0,3729/2021,2021-05-25T00:00:00,Green Asparagus,UK,Aberdeen & Stanton Ltd,"67 New Spitalfields Market, Sherrin Road, Leyt...",Watts Farms,"Watts Farm Farningham Hill Road, Farningham, K...",None were detected above the set RL
1,3697/2021,2021-05-25T00:00:00,Asparagus,UK,Addey & Son (Western International) Ltd,"P35 Western International Market, Hayes Road, ...",Spanton Farms,"Edward Spanton Farms Wayborough Farm, Minster,...",None were detected above the set RL
2,3715/2021,2021-05-11T00:00:00,Asparagus,UK,Aldi,"Thomsett Way, Queenborough, Kent ME11 5AR",Aldi,,None were detected above the set RL
3,5503/2021,2021-11-15T00:00:00,Asparagus Tips,Peru,Aldi,"Marton Road, Middlesbrough TS4 2PQ",Nature's Pick,"Aldi Stores Ltd PO Box 26. Atherstone, Warwick...",None were detected above the set RL
4,0565/2021,2021-10-18T00:00:00,Asparagus,Peru,Asda,"Western Way, Bury St Edmunds IP33 3SP",ASDA Grower Selection,"ASDA Stores Ltd Great Wilson Street, Leeds LS1...",None were detected above the set RL
...,...,...,...,...,...,...,...,...,...
38007,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",azoxystrobin 0.08 (MRL = 3)
38008,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",boscalid 0.4 (MRL = 3)
38009,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",fenpyrazamine 0.3 (MRL = 3)
38010,0370/2022 Organic,2022-03-14T00:00:00,Organic Ramyle,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5HD",Tesco Organic,"Tesco Ireland Ltd Gresham House, Marine Road, ...",None were detected above the set RL


In [182]:
df.isna().sum()

Sample ID                                  0
Date of Sampling                           0
Description                                0
Country of Origin                          0
Sampling Point                             0
Address                                    0
Brand Name                                 0
Packer / Manufacturer / Importer           0
Pesticide residues found in mg/kg (MRL)    0
dtype: int64

In [183]:
renaming_dict ={
old_name : lowercase_name 
    for old_name,lowercase_name 
    in zip(list(df),[new_name.lower().replace(' ','_') 
                     for new_name in list(df)])
}
renaming_dict.update({'Packer / Manufacturer / Importer': 'packer / manufacturer / importer'})
df = df.rename(columns=renaming_dict)

In [185]:
df.replace({'None were detected above the set RL': 'n/a'}, regex=True, inplace=True)
df

,sample_id,date_of_sampling,description,country_of_origin,sampling_point,address,brand_name,packer / manufacturer / importer,pesticide_residues_found_in_mg/kg_(mrl)
0,3729/2021,2021-05-25T00:00:00,Green Asparagus,UK,Aberdeen & Stanton Ltd,"67 New Spitalfields Market, Sherrin Road, Leyt...",Watts Farms,"Watts Farm Farningham Hill Road, Farningham, K...",n/a
1,3697/2021,2021-05-25T00:00:00,Asparagus,UK,Addey & Son (Western International) Ltd,"P35 Western International Market, Hayes Road, ...",Spanton Farms,"Edward Spanton Farms Wayborough Farm, Minster,...",n/a
2,3715/2021,2021-05-11T00:00:00,Asparagus,UK,Aldi,"Thomsett Way, Queenborough, Kent ME11 5AR",Aldi,,n/a
3,5503/2021,2021-11-15T00:00:00,Asparagus Tips,Peru,Aldi,"Marton Road, Middlesbrough TS4 2PQ",Nature's Pick,"Aldi Stores Ltd PO Box 26. Atherstone, Warwick...",n/a
4,0565/2021,2021-10-18T00:00:00,Asparagus,Peru,Asda,"Western Way, Bury St Edmunds IP33 3SP",ASDA Grower Selection,"ASDA Stores Ltd Great Wilson Street, Leeds LS1...",n/a
...,...,...,...,...,...,...,...,...,...
38007,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",azoxystrobin 0.08 (MRL = 3)
38008,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",boscalid 0.4 (MRL = 3)
38009,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",fenpyrazamine 0.3 (MRL = 3)
38010,0370/2022 Organic,2022-03-14T00:00:00,Organic Ramyle,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5HD",Tesco Organic,"Tesco Ireland Ltd Gresham House, Marine Road, ...",n/a


In [186]:
df['pesticide_residues_found_in_mg/kg_(mrl)'].head(20) # looking at the data in this column


0                                    n/a
1                                    n/a
2                                    n/a
3                                    n/a
4                                    n/a
5                                    n/a
6     flonicamid (sum) 0.2 (MRL = 0.03*)
7                                    n/a
8                                    n/a
9                                    n/a
10                                   n/a
11                                   n/a
12                                   n/a
13                                   n/a
14                                   n/a
15                                   n/a
16                                   n/a
17                                   n/a
18                                   n/a
19                                   n/a
Name: pesticide_residues_found_in_mg/kg_(mrl), dtype: object

In [187]:
df2=df['pesticide_residues_found_in_mg/kg_(mrl)'].str.extract(r'(.*)\s(\d[\d.]*)\s+\(MRL\s*=\s*(\d[\d.]*)\)')
df2.head(20)

,0,1,2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [188]:
df = pd.concat([df,df2], axis = 1, sort = False) #new columns are currently 0,1,2
df.rename(columns={0:'chem_name',1:'amount_detected',2:'mrl'},inplace=True) # rename columns 0,1,2 to chem_name, amount_detected, and mrl
df.drop("pesticide_residues_found_in_mg/kg_(mrl)",1, inplace=True) # and deleted the old combined column
df

C:\Users\gabes\AppData\Local\Temp\ipykernel_1208\3238331631.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop("pesticide_residues_found_in_mg/kg_(mrl)",1, inplace=True) # and deleted the old combined column


,sample_id,date_of_sampling,description,country_of_origin,sampling_point,address,brand_name,packer / manufacturer / importer,chem_name,amount_detected,mrl
0,3729/2021,2021-05-25T00:00:00,Green Asparagus,UK,Aberdeen & Stanton Ltd,"67 New Spitalfields Market, Sherrin Road, Leyt...",Watts Farms,"Watts Farm Farningham Hill Road, Farningham, K...",NaN,NaN,NaN
1,3697/2021,2021-05-25T00:00:00,Asparagus,UK,Addey & Son (Western International) Ltd,"P35 Western International Market, Hayes Road, ...",Spanton Farms,"Edward Spanton Farms Wayborough Farm, Minster,...",NaN,NaN,NaN
2,3715/2021,2021-05-11T00:00:00,Asparagus,UK,Aldi,"Thomsett Way, Queenborough, Kent ME11 5AR",Aldi,,NaN,NaN,NaN
3,5503/2021,2021-11-15T00:00:00,Asparagus Tips,Peru,Aldi,"Marton Road, Middlesbrough TS4 2PQ",Nature's Pick,"Aldi Stores Ltd PO Box 26. Atherstone, Warwick...",NaN,NaN,NaN
4,0565/2021,2021-10-18T00:00:00,Asparagus,Peru,Asda,"Western Way, Bury St Edmunds IP33 3SP",ASDA Grower Selection,"ASDA Stores Ltd Great Wilson Street, Leeds LS1...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
38007,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",azoxystrobin,0.08,3
38008,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",boscalid,0.4,3
38009,0005/2022,2022-01-24T00:00:00,Julita,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5XD",Tesco Finest,"Tesco Stores Ltd Tesco House, Shire Park, Kest...",fenpyrazamine,0.3,3
38010,0370/2022 Organic,2022-03-14T00:00:00,Organic Ramyle,Spain,Tesco,"Jubilee Way, Newtownabbey BT36 5HD",Tesco Organic,"Tesco Ireland Ltd Gresham House, Marine Road, ...",NaN,NaN,NaN


In [96]:
df.count()

sample_id                           161
date_of_sampling                    161
description                         161
country_of_origin                   161
sampling_point                      161
address                             161
brand_name                          161
packer_/_manufacturer_/_importer    161
chem_name                           133
amount_detected                     133
mrl                                 133
dtype: int64

In [97]:
df['mrl'] = pd.to_numeric(df['mrl']) #converting mrls and amounts detected to numeric values
df['amount_detected'] = pd.to_numeric(df['amount_detected'])
df['amount_detected'].mean()

0.1274812030075186

In [101]:
df.groupby(['sample_id','description','chem_name']).sum()

amount_detected    mrl
sample_id description     chem_name                                
0074/2021 Aubergine       Fosetyl (sum)                0.800  100.0
                          fluopyram                    0.020    0.9
                          spiromesifen                 0.010    0.5
0093/2021 Aubergine       chlorate                     1.600    0.4
0183/2021 Baby Aubergines cyprodinil                   0.050    1.5
...                                                      ...    ...
4251/2021 Aubergine       spinetoram                   0.010    0.5
4256/2021 Baby Aubergine  carbendazim (sum)            0.004    0.5
4752/2021 Aubergine       chlorate                     0.010    0.4
                          fluopyram                    0.020    0.9
5886/2021 Aubergines      tebuconazole                 0.030    0.4

[132 rows x 2 columns]